In [23]:
import numpy as np
import pandas as pd

In [37]:
def dispersion_col(df_train, df_test, col_name):
    if (df_train[col_name].dtype != 'O'):
#         print(col_name + "is number")
        return
    g = df_train.groupby(col_name)
    if len(g.size()) > 200:
#         print(col_name + "group to big")
        return
    
    for name, group in g:
        new_col_name = col_name + name
        df_train[new_col_name] = 0
        df_test[new_col_name] = 0
    
    for name, group in g:
        new_col_name = col_name + name
        df_train.loc[df_train[col_name] == name, new_col_name] = 1
        df_test.loc[df_test[col_name] == name, new_col_name] = 1
        
    
#     df.drop([col_name], axis=1)

In [38]:
train = pd.read_csv('C:\\Users\\Chao\\Documents\\DMProject\\BNP\\input\\train.csv')
test = pd.read_csv('C:\\Users\\Chao\\Documents\\DMProject\\BNP\\input\\test.csv')
target = train['target']
id_test = test['ID'].values

In [39]:
train = train.drop(['ID', 'target'], axis=1)
test = test.drop(['ID'], axis=1)
column_names = train.columns

In [40]:
# dispersion_col(train, 'v3')

for col_name in column_names:
    dispersion_col(train, test, col_name)

In [41]:
train = train.drop(['v3', 'v22', 'v24', 'v30', 'v31', 
                    'v47', 'v52', 'v56', 'v66', 'v71', 
                    'v74', 'v75', 'v79', 'v91', 
                    'v107', 'v110', 'v112', 'v113', 'v125'], axis=1)
test = test.drop(['v3', 'v22', 'v24', 'v30', 'v31', 
                    'v47', 'v52', 'v56', 'v66', 'v71', 
                    'v74', 'v75', 'v79', 'v91', 
                    'v107', 'v110', 'v112', 'v113', 'v125'], axis=1)


In [42]:
future = train.columns.values

In [43]:
from sklearn.preprocessing import Imputer
imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
imp.fit(train)

Imputer(axis=0, copy=True, missing_values='NaN', strategy='mean', verbose=0)

In [44]:
train_imputed = imp.transform(train)
test_imputed = imp.transform(test)

In [45]:
from sklearn.linear_model import LogisticRegression
from sklearn import cross_validation
from sklearn import linear_model

In [52]:
alg    = LogisticRegression(random_state=1)
alg.fit(train_imputed, target)
# scores = cross_validation.cross_val_score(
#     alg,
#     train_imputed[future],
#     target,
#     cv=3.
# )

# print(scores.mean())

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=1, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [47]:
y_pred = alg.predict_proba(test_imputed)
len(y_pred)

114393

In [48]:
y_value = alg.predict(test_imputed)

In [49]:
#print y_pred

pd.DataFrame({"ID": id_test, "PredictedProb": y_pred[:,1]}).to_csv('lr_dispression.csv',index=False)